# Advanced Simulated Tool Calling with Open Models

This notebook demonstrates advanced patterns for simulating function (tool) calling with open models using the OpenAI Python SDK. 

Unlike the basic examples, these scenarios require the model to output an *array* of JSON objects (for multiple function calls in a single response, chained reasoning, or batch calls). 

Because the OpenAI SDK's `response_format={"type": "json_object"}` only supports a single JSON object (not an array), we **do not** use `response_format` here. Instead, we rely on careful prompt engineering to instruct the model to output a valid JSON array.

## Prerequisites
1. Sign up for an account at [Featherless](https://featherless.ai/register)
2. Subscribe to a plan and get your API key from [API Keys](https://featherless.ai/account/api-keys)

## Setup
First, import the required libraries and set up your API key.

In [5]:
# Install the OpenAI Python SDK if you haven't already
# !pip install openai

In [6]:
from openai import OpenAI

# Initialize the OpenAI client with your endpoint and API key
client = OpenAI(
    base_url="https://api.featherless.ai/v1",
    api_key="YOUR FEATHERLESS API KEY",
)

---
## Example 4: Multiple Function Calls in One Response

In this example, the model is instructed to output an **array** of function calls (e.g., both `get_weather` and `get_time` for multiple cities). 

**Note:** We do *not* use `response_format={"type": "json_object"}` because the response is a JSON array, not a single object. Instead, we rely on prompt instructions to ensure the output is a valid JSON array.

In [7]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to two functions:\n"
                '1. get_weather(location: str)\n'
                '2. get_time(location: str)\n'
                "When asked a question, respond ONLY in the following JSON format:\n"
                '[\n'
                '  {"function": "get_weather", "arguments": {"location": "<city, country>"}},\n'
                '  {"function": "get_time", "arguments": {"location": "<city, country>"}},\n'
                '  ...\n'
                ']\n'
                "If only one function is needed, return a list with one item. "
                "Do not answer the question directly. Only output the JSON array."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather and time in London and Paris?"
        }
    ]
)
print("Simulated Tool Calling (Multiple Calls) Output:", response.choices[0].message.content)

Simulated Tool Calling (Multiple Calls) Output: [
  {"function": "get_weather", "arguments": {"location": "London, UK"}},
  {"function": "get_time", "arguments": {"location": "London, UK"}},
  {"function": "get_weather", "arguments": {"location": "Paris, France"}},
  {"function": "get_time", "arguments": {"location": "Paris, France"}}
]


---
## Example 5: Chained Reasoning (Dependent Function Calls)

This example demonstrates *chained reasoning*: the model must first call `get_location_of_event` to get a location, then use that result as input to `get_weather`. 

Again, the output is a JSON array, so we do not use `response_format`.

In [8]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to two functions:\n"
                '1. get_location_of_event(event: str) -> location: str\n'
                '2. get_weather(location: str)\n'
                "If the user asks about the weather for an event, first call get_location_of_event, "
                "then use its output as the location argument for get_weather. "
                "Respond ONLY in the following JSON format:\n"
                '[\n'
                '  {"function": "get_location_of_event", "arguments": {"event": "<event_name>"}},\n'
                '  {"function": "get_weather", "arguments": {"location": "<location_from_previous_call>"}},\n'
                ']\n'
                "Do not answer the question directly. Only output the JSON array."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather like at the Wimbledon tennis final?"
        }
    ]
)
print("Simulated Tool Calling (Chained Reasoning) Output:", response.choices[0].message.content)

Simulated Tool Calling (Chained Reasoning) Output: [
  {"function": "get_location_of_event", "arguments": {"event": "Wimbledon tennis final"}},
  {"function": "get_weather", "arguments": {"location": "<location_from_previous_call>"}},
]


---
## Example 6: Function with Multiple Arguments

This example shows how to simulate a function that takes multiple arguments. The output is a single JSON object, so you *could* use `response_format`, but here we show the prompt-based approach for consistency.

In [9]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to a function:\n"
                'get_flight_status(flight_number: str, date: str)\n'
                "When asked, respond ONLY in the following JSON format:\n"
                '{\n  "function": "get_flight_status",\n  "arguments": {"flight_number": "<flight_number>", "date": "<YYYY-MM-DD>"}\n}\n'
                "Do not answer the question directly. Only output the JSON."
            ),
        },
        {
            "role": "user",
            "content": "What's the status of flight BA123 on June 10th?"
        }
    ]
)
print("Simulated Tool Calling (Multiple Arguments) Output:", response.choices[0].message.content)

Simulated Tool Calling (Multiple Arguments) Output: {
  "function": "get_flight_status",
  "arguments": {"flight_number": "BA123", "date": "2024-06-10"}
}


---
## Example 7: Batch Calls for Multiple Cities

This example demonstrates how to ask the model to output a JSON array of function calls for multiple cities in a single response. Again, we do not use `response_format`.

In [10]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to a function:\n"
                'get_weather(location: str)\n'
                "When asked about multiple cities, respond ONLY in the following JSON format:\n"
                '[\n'
                '  {"function": "get_weather", "arguments": {"location": "<city1, country1>"}},\n'
                '  {"function": "get_weather", "arguments": {"location": "<city2, country2>"}},\n'
                '  ...\n'
                ']\n'
                "Do not answer the question directly. Only output the JSON array."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather in Berlin, Madrid, and Rome?"
        }
    ]
)
print("Simulated Tool Calling (Batch Calls) Output:", response.choices[0].message.content)

Simulated Tool Calling (Batch Calls) Output: [
  {"function": "get_weather", "arguments": {"location": "Berlin, Germany"}},
  {"function": "get_weather", "arguments": {"location": "Madrid, Spain"}},
  {"function": "get_weather", "arguments": {"location": "Rome, Italy"}}
]


---
## Summary

- For advanced tool calling scenarios (multiple calls, chained reasoning, batch calls), instruct the model to output a JSON array.
- Do **not** use `response_format={"type": "json_object"}` when expecting a JSON array, as it only supports single objects.
- Use clear prompt engineering to ensure the model outputs valid JSON arrays for downstream parsing.

---

---
## Example 8: Named JSON List with `response_format={"type": "json_object"}`

If you want to return multiple function calls but still use the `response_format={"type": "json_object"}` parameter, you can instruct the model to return a *named* list (e.g., `{ "calls": [ ... ] }`).

This works because the top-level structure is a JSON object, not an array. This is a good compromise if you want to leverage OpenAI’s response validation for JSON objects, but still need to return multiple calls.

In [11]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to two functions:\n"
                '1. get_weather(location: str)\n'
                '2. get_time(location: str)\n'
                "When asked a question, respond ONLY in the following JSON format:\n"
                '{\n  "calls": [\n    {"function": "get_weather", "arguments": {"location": "<city, country>"}},\n    {"function": "get_time", "arguments": {"location": "<city, country>"}}\n  ]\n}\n'
                "If only one function is needed, return a list with one item. "
                "Do not answer the question directly. Only output the JSON object."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather and time in London and Paris?"
        }
    ],
    response_format={"type": "json_object"},
)
print("Simulated Tool Calling (Named JSON List) Output:", response.choices[0].message.content)

Simulated Tool Calling (Named JSON List) Output: {
  "calls": [
    {"function": "get_weather", "arguments": {"location": "London, UK"}},
    {"function": "get_time", "arguments": {"location": "London, UK"}},
    {"function": "get_weather", "arguments": {"location": "Paris, France"}},
    {"function": "get_time", "arguments": {"location": "Paris, France"}}
  ]
}


---
## Example 9: Chained Reasoning with Named JSON List

You can also use a named list for chained reasoning scenarios. This allows you to keep the output as a valid JSON object for the SDK, while still representing multiple dependent calls.

In [12]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to two functions:\n"
                '1. get_location_of_event(event: str) -> location: str\n'
                '2. get_weather(location: str)\n'
                "If the user asks about the weather for an event, first call get_location_of_event, "
                "then use its output as the location argument for get_weather. "
                "Respond ONLY in the following JSON format:\n"
                '{\n  "calls": [\n    {"function": "get_location_of_event", "arguments": {"event": "<event_name>"}},\n    {"function": "get_weather", "arguments": {"location": "<location_from_previous_call>"}}\n  ]\n}\n'
                "Do not answer the question directly. Only output the JSON object."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather like at the Wimbledon tennis final?"
        }
    ],
    response_format={"type": "json_object"},
)
print("Simulated Tool Calling (Chained Reasoning, Named List) Output:", response.choices[0].message.content)

Simulated Tool Calling (Chained Reasoning, Named List) Output: {
  "calls": [
    {"function": "get_location_of_event", "arguments": {"event": "Wimbledon tennis final"}},
    {"function": "get_weather", "arguments": {"location": "<location_from_previous_call>"}}
  ]
}


---
## Example 10: Batch Calls for Multiple Cities with Named JSON List

This example demonstrates how to ask the model to output a named JSON list of function calls for multiple cities, while still using `response_format={"type": "json_object"}`.

In [13]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant. "
                "You have access to a function:\n"
                'get_weather(location: str)\n'
                "When asked about multiple cities, respond ONLY in the following JSON format:\n"
                '{\n  "calls": [\n    {"function": "get_weather", "arguments": {"location": "<city1, country1>"}},\n    {"function": "get_weather", "arguments": {"location": "<city2, country2>"}}\n  ]\n}\n'
                "Do not answer the question directly. Only output the JSON object."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather in Berlin, Madrid, and Rome?"
        }
    ],
    response_format={"type": "json_object"},
)
print("Simulated Tool Calling (Batch Calls, Named List) Output:", response.choices[0].message.content)

Simulated Tool Calling (Batch Calls, Named List) Output: {
  "calls": [
    {"function": "get_weather", "arguments": {"location": "Berlin, Germany"}},
    {"function": "get_weather", "arguments": {"location": "Madrid, Spain"}},
    {"function": "get_weather", "arguments": {"location": "Rome, Italy"}}
  ]
}
